# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [3]:
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [4]:
orders = pd.read_csv("Orders.csv")
print(orders.head())

   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [5]:
# Here, I have decided to create a new dataframe specifically to obtain the aggregated amount spent by each individual client.

# New agg_amount_spent df:

agg_amount_spent = orders.groupby('CustomerID')['amount_spent'].sum().reset_index()

print(agg_amount_spent)



      CustomerID  amount_spent
0          12346      77183.60
1          12347       4310.00
2          12348       1797.24
3          12349       1757.55
4          12350        334.40
...          ...           ...
4334       18280        180.60
4335       18281         80.82
4336       18282        178.05
4337       18283       2094.88
4338       18287       1837.28

[4339 rows x 2 columns]


In [6]:
import pandas as pd

quantiles = [0, 0.75, 0.95, 1]

agg_amount_spent['quantile'] = pd.qcut(agg_amount_spent['amount_spent'], q=quantiles, labels=['below_75', 'between_75_95', 'above_95'])

quantile_counts = agg_amount_spent.groupby(['quantile', 'CustomerID']).size().unstack('quantile', fill_value=0)

print(quantile_counts)




quantile    below_75  between_75_95  above_95
CustomerID                                   
12346              0              0         1
12347              0              1         0
12348              0              1         0
12349              0              1         0
12350              1              0         0
...              ...            ...       ...
18280              1              0         0
18281              1              0         0
18282              1              0         0
18283              0              1         0
18287              0              1         0

[4339 rows x 3 columns]


In [7]:
import pandas as pd

quantile_counts = quantile_counts.reset_index()

Labels = pd.DataFrame(index=quantile_counts.index)
Labels['CustomerID'] = quantile_counts['CustomerID']

def label_customer(row):
    if row['below_75'] == 1:
        return 'Regular'
    elif row['between_75_95'] == 1:
        return 'Preferred'
    elif row['above_95'] == 1:
        return 'VIP'
    else:
        return 'Unknown'

Labels['Label'] = quantile_counts.apply(label_customer, axis=1)

print(Labels)







      CustomerID      Label
0          12346        VIP
1          12347  Preferred
2          12348  Preferred
3          12349  Preferred
4          12350    Regular
...          ...        ...
4334       18280    Regular
4335       18281    Regular
4336       18282    Regular
4337       18283  Preferred
4338       18287  Preferred

[4339 rows x 2 columns]


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [11]:
merged_df = pd.merge(orders, Labels, on='CustomerID')

result_df = merged_df[['CustomerID', 'Country', 'Label']]

print(result_df.head())

print()

vip_counts = result_df[result_df['Label'] == 'VIP'].groupby('Country').size()

most_vip_country = vip_counts.idxmax()

print(f"The country with the most VIP customers is {most_vip_country}.")



   CustomerID         Country      Label
0       17850  United Kingdom  Preferred
1       17850  United Kingdom  Preferred
2       17850  United Kingdom  Preferred
3       17850  United Kingdom  Preferred
4       17850  United Kingdom  Preferred

The country with the most VIP customers is United Kingdom.


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [9]:
top_counts = result_df[result_df['Label'].isin(['VIP', 'Preferred'])].groupby('Country').size()

most_top_country = vip_counts.idxmax()

print(f"The country with the most VIP+Preferred Customers combined is {most_top_country}.")

The country with the most VIP+Preferred Customers combined is United Kingdom.
